In [ ]:
import torch
import numpy as np
import os
from torchvision.io import read_image
import os
import matplotlib.pyplot as plt
import scipy.io
import h5py
import cv2
from spectral import *
from utils import Loss_Fid, Loss_SAM, Loss_SSIM, reconRGB, bgr2rgb
import hyperspy.api as hs

cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
ROOT = '/work3/s212645/Spectral_Reconstruction'

TRAIN_RGB = '/Train_RGB/'
TRAIN_SP = '/Valid_Spec/'
device

In [ ]:
import scipy.io
import re

spec = '/work3/s212645/Spectral_Reconstruction/CAVE/'

def Rename(spec):
    filelist = os.listdir(spec)
    reg = re.compile(r'.*.mat')
    for file in filelist:
        if re.findall(reg, file):
            oldname = spec + file
            number = file.split('.mat')[0]
            newnumber = number.zfill(3)
            newname = spec + newnumber + '.mat'
            os.rename(oldname, newname)

filelist = os.listdir(spec)
filelist.sort()
reg = re.compile(r'.*.mat')
for file in filelist:
    if re.match(reg, file):
        mat = scipy.io.loadmat(spec+file)
        hyper = np.float32(np.array(mat['cube']))
        rgb = np.float32(np.array(mat['rgb']))
        break

In [ ]:
import torch.nn as nn
from torchmetrics.image.fid import FrechetInceptionDistance
class Loss_Fid(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fid = FrechetInceptionDistance(feature=2048)
        
    def forward(self, outputs, label):
        outputs = outputs * 255
        label = label * 255
        outputs = outputs.type(torch.ByteTensor)
        label = label.type(torch.ByteTensor)
        self.fid.update(label, real=True)
        self.fid.update(outputs, real=False)
        fidscore = self.fid.compute()
        return fidscore

In [ ]:
with h5py.File(spec, 'r') as mat:
    hyper =np.float32(np.array(mat['cube']))
hyper = np.transpose(hyper, [2, 1, 0])
subhyper = cv2.resize(hyper, [128, 128])
print(subhyper.shape)
rgb = get_rgb(subhyper, (29, 19, 9)) # fid score
plt.imshow(rgb)
# indices = [22, 12, 3]
# subhyper = hyper[indices]
# subhyper = np.transpose(subhyper, [2, 1, 0])
# subhyper = cv2.cvtColor(subhyper, cv2.COLOR_BGR2RGB)
# subhyper = np.float32(subhyper)
# subhyper = (subhyper-subhyper.min())/(subhyper.max()-subhyper.min())
# plt.imshow(subhyper)
# view = ImageView()
# view.set_data(hyper, (29, 19, 9)) # fid score
# # view.set_data(hyper, (20, 13, 9)) # sam
# plt.imshow(view.data_rgb)
# subhyper = torch.tensor(view.data_rgb.transpose(2, 0, 1), dtype=torch.float32).to(device)
# sam = Loss_SAM().to(device)
# fid = Loss_Fid().to(device)
# lossmin = 1e12
# argi = 0
# argj = 1
# argk = 2

# loss = fid(bgr.reshape(8, 3, 482 // 2, 512 // 4), subhyper.reshape(8, 3, 482 // 2, 512 // 4))
# # print(subhyper)
# for i in range(0,10):
#     for j in range(10,20):
#         for k in range(20, 31):
#             view.set_data(hyper, (k, j, i))
#             subhyper = torch.tensor(view.data_rgb.transpose(2, 0, 1), dtype=torch.float32).to(device)
#             loss = fid(bgr.reshape(8, 3, 482 // 2, 512 // 4), subhyper.reshape(8, 3, 482 // 2, 512 // 4))
#             if loss < lossmin:
#                 lossmin = loss
#                 argi = i
#                 argj = j
#                 argk = k
#                 print(loss)

In [ ]:
path = '/work3/s212645/Spectral_Reconstruction/ICVL/data/mat/4cam_0411-1640-1.mat'
with h5py.File(path, 'r') as mat:
    hyper = np.float32(np.array(mat['rad']))
    rgb = np.float32(np.array(mat['rgb']))
hyper = np.transpose(hyper, [1, 2, 0])
hyper = np.flip(hyper, 0)
rgb = np.transpose(rgb, [2, 1, 0])
rgb = np.flip(rgb, 2)
# view = ImageView()
rgb = get_rgb(hyper, (29, 19, 9)) # fid score
plt.imshow(rgb)

In [ ]:
plt.imshow(rgb)

In [ ]:
names = []
for path, dirs, files in os.walk(ROOT+TRAIN_SP):
    for file in files:
        num_video = os.path.basename(path)
        names.append(os.path.basename(os.path.dirname(path)))
        hyper_path = os.path.join(path,file)
        with h5py.File(hyper_path, 'r') as mat:
            hyper =np.float32(np.array(mat['cube']))
        hyper = np.transpose(hyper, [2, 1, 0])
        imshow(hyper, (22, 10, 3))
        break

In [ ]:
names = []
for path, dirs, files in os.walk(ROOT+TRAIN_RGB):
    for file in files:
        num_video = os.path.basename(path)
        names.append(os.path.basename(os.path.dirname(path)))
        hyper_path = os.path.join(path,file)
        bgr_path = os.path.join(path,file)
        bgr = cv2.imread(bgr_path)
        bgr2rgb = True
        if bgr2rgb:
            bgr = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        bgr = np.float32(bgr)
        bgr = (bgr-bgr.min())/(bgr.max()-bgr.min())
        bgr = np.transpose(bgr, [2, 0, 1])  # [3,482,512]
        
        stride = 8
        crop_size = 128
        idx = 0
        
        h,w = 482,512  # img shape
        patch_per_line = (w-crop_size)//stride+1
        patch_per_colum = (h-crop_size)//stride+1
        patch_per_img = patch_per_line*patch_per_colum
        img_idx, patch_idx = idx//patch_per_img, idx%patch_per_img
        h_idx, w_idx = patch_idx//patch_per_line, patch_idx%patch_per_line
        img = bgr[:,h_idx*stride:h_idx*stride+crop_size, w_idx*stride:w_idx*stride+crop_size]
        
        img = img.transpose(1,2,0)
        print(img.shape)
        plt.imshow(bgr.transpose(1,2,0))
        
        break

In [ ]:
outputs = torch.randn([64,31,128,128])
label = torch.randn_like(outputs)
error = torch.abs(outputs.view(-1) - label.view(-1)) / label.view(-1)
# mrae = torch.mean(error.reshape(-1))
mrae = torch.mean(error.view(-1))
mrae
a = np.array([0,1,1])
print(a.all())

In [ ]:
import torch.nn.functional as F
recons = torch.randn(1, 31, 128, 128)
input = torch.randn_like(recons)
mu = torch.randn(1, 1024)
log_var = torch.randn_like(mu)

kld_weight = 1024 / 128 / 128 / 31 # Account for the minibatch samples from the dataset
recons_loss =F.mse_loss(recons, input)

kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)
print(recons_loss, kld_loss * kld_weight) 

In [ ]:
import torch

import matplotlib.pyplot as plt


def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size

    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    return windows


window_size = 4
shift_size = 3
H, W = 16, 16
img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
h_slices = (slice(0, -window_size),
            slice(-window_size, -window_size * 2),
            slice(-window_size * 2, -window_size * 3),
            slice(-window_size * 3, -shift_size),
            slice(-shift_size, None))
w_slices = (slice(0, -window_size),
            slice(-window_size, -window_size * 2),
            slice(-window_size * 2, -window_size * 3),
            slice(-window_size * 3, -shift_size),
            slice(-shift_size, None))
cnt = 0
for h in h_slices:
    for w in w_slices:
        img_mask[:, h, w, :] = cnt
        cnt += 1

mask_windows = window_partition(img_mask, window_size)  # nW, window_size, window_size, 1
mask_windows = mask_windows.view(-1, window_size * window_size)

attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))

plt.matshow(img_mask[0, :, :, 0].numpy())
for i in range(16):
    plt.matshow(attn_mask[i].numpy())

plt.show()

In [1]:
import sys
sys.path.append('./')
from torch.utils.data import Dataset
import numpy as np
import random
import cv2
import h5py
from torch.utils.data import DataLoader
from torch.autograd import Variable
import os
import torch
import torch.nn as nn
# from options import opt
import scipy.io
import re
import matplotlib.pyplot as plt

root = '/work3/s212645/Spectral_Reconstruction/'
datanames = ['ICVL/', 'ARAD/', 'CAVE/']

def Rename(path):
    filelist = os.listdir(path)
    reg = re.compile(r'.*.mat')
    for file in filelist:
        if re.findall(reg, file):
            oldname = path + file
            number = file.split('.mat')[0]
            newnumber = number.zfill(3)
            newname = path + newnumber + '.mat'
            os.rename(oldname, newname)

def gen_test_list(matlist, startidx, size):
    testlist = []
    for i in range(size):
        testlist.append(matlist[i + startidx])
    return testlist

def cross_validation_lists(path, test_size):
    crossvaltrainlists = []
    crossvaltestlists = []
    filelist = os.listdir(path)
    filelist.sort()
    assert len(filelist) > test_size
    reg = re.compile(r'.*.mat')
    matlist = []
    for file in filelist:
        if re.match(reg, file):
            matlist.append(file)
    matlist.sort()
    all_length = len(matlist)
    assert all_length % test_size == 0
    for i in range(all_length // test_size):
        trainlist = []
        testlist = []
        testlist = gen_test_list(matlist, i * test_size, test_size)
        trainlist = matlist.copy()
        for mat in testlist:
            trainlist.remove(mat)
        crossvaltestlists.append(testlist)
        crossvaltrainlists.append(trainlist)
    return crossvaltrainlists, crossvaltestlists

def cross_validation_dataset(path, test_size, imsize):
    train_sets = []
    test_sets = []
    crossvaltrainlists, crossvaltestlists = cross_validation_lists(path, test_size)
    num_crossval = len(crossvaltrainlists)
    for i in range(num_crossval):
        train_sets.append(get_dataset(crossvaltrainlists[i], imsize))
        test_sets.append(get_dataset(crossvaltestlists[i], imsize))
    return train_sets, test_sets

def get_dataset(list, imsize):
    specs, rgbs = get_all_mats(list, imsize)
    return specs, rgbs

def split_train_test(path, test_size):
    filelist = os.listdir(path)
    filelist.sort()
    assert len(filelist) > test_size
    reg = re.compile(r'.*.mat')
    matlist = []
    for file in filelist:
        if re.match(reg, file):
            matlist.append(file)
    all_length = len(matlist)
    train_size = all_length - test_size
    matlist.sort()
    trainlist = []
    testlist = []
    for i in range(train_size):
        trainlist.append(matlist[i])
    for j in range(test_size):
        testlist.append(matlist[j + train_size])
    return trainlist, testlist

def get_all_mats(filelist, imsize):
    specs = []
    rgbs = []
    for file in filelist:
        mat = scipy.io.loadmat(path+file)
        spec, rgb = get_all_patches(mat, imsize)
        specs += spec
        rgbs += rgb
        # break
    return specs, rgbs
    
def Resize(hyper, rgb, h, w):
    hyper_s = cv2.resize(hyper, [h, w])
    rgb_s = cv2.resize(rgb, [h, w])
    return hyper_s, rgb_s

def data_resize(mat, imsize):
    spectral_images = []
    rgb_images = []
    hyper = np.float32(np.array(mat['cube']))
    rgb = np.float32(np.array(mat['rgb']))
    h = rgb.shape[0]
    w = rgb.shape[1]
    while min(h // imsize, w // imsize) >= 1:
        hyper_s, rgb_s = Resize(hyper, rgb, h, w)
        spectral_images.append(hyper_s)
        rgb_images.append(rgb_s)
        h = h // 2
        w = w // 2
    if h > imsize / 2 or w > imsize / 2 :
        hyper_s, rgb_s = Resize(hyper, rgb, imsize, imsize)
        spectral_images.append(hyper_s)
        rgb_images.append(rgb_s)
    return spectral_images, rgb_images

def patch_gen(array, imsize, h, w):
    return array[h-imsize:h, w-imsize:w, :]

def patch_image(array, imsize):
    h = array.shape[0]
    w = array.shape[1]
    assert h >= imsize
    assert w >= imsize
    iter_h = int(np.ceil(h / imsize) + 1)
    iter_w = int(np.ceil(w / imsize) + 1)
    patches = []
    for i in range(1, iter_h):
        for j in range(1, iter_w):
            patches.append(patch_gen(array, imsize, min(h, i * imsize), min(w, j * imsize)))
    return patches
    
def get_all_patches(mat, imsize):
    spectrals = []
    rgbs = []
    resize_spectrals, resize_rgbs = data_resize(mat, imsize)
    for i in range(len(resize_spectrals)):
        hyperpatches = patch_image(resize_spectrals[i], imsize)
        rgbpatches = patch_image(resize_rgbs[i], imsize)
        spectrals += hyperpatches
        rgbs += rgbpatches
    return spectrals, rgbs

if __name__ == '__main__':
    path = root + datanames[2]
    # spectral_images, rgb_images = get_all_mats(path, 128)
    # print(len(rgb_images))
    # print(rgb_images[-1].shape)
    train_sets, test_sets = cross_validation_dataset(path, 1, 128)
    print(len(train_sets[0]))
    print(sys.getsizeof(train_sets[0][0]))
    imshow(train_sets[0][1][-1])

MemoryError: 